# **Consensus atlas of ATAC regions (hg38)** 

In [ ]:
import pandas as pd

import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

import os

# pip install gprofiler
from gprofiler import gprofiler

# **ATAC regulatory regions from human brain primary samples**

In [ ]:
pwd

In [ ]:
!mkdir /home/jovyan/jm_jlab/data_indNeuro/consensus_atlas_ATACregions_hg38/

In [ ]:
folder = "/home/jovyan/jm_jlab/data_indNeuro/consensus_atlas_ATACregions_hg38/"

In [ ]:
cd $folder

### **Trevino all (hg38)**

In [ ]:
# From GitHub: https://github.com/GreenleafLab/brainchromatin/blob/main/links.txt

#Multiome:
!wget https://atrev.s3.amazonaws.com/brainchromatin/multiome_atac_consensus_peaks.txt.gz -O tr21multiome.txt.gz

#single ATAC-seq
!wget https://atrev.s3.amazonaws.com/brainchromatin/atac_consensus_peaks.bed.gz -O tr21atac.bed.gz

**Markenscoff-Papadimitriou et al (hg38)**

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE149nnn/GSE149268/suppl/GSE149268%5Fannotation%2Docr%2Dhg38%2Ebed%2Egz -O mk20ocr.txt.gz

**de la Torre-Ubieta et. al (hg19)**

In [ ]:
!wget https://www.cell.com/cms/10.1016/j.cell.2017.12.014/attachment/ec8faf3a-8470-4855-886d-cfb014c48573/mmc1.xlsx -O dT.xlsx

**Processing**

In [ ]:
ls

In [ ]:
mk20 = pd.read_csv("mk20ocr.txt.gz", sep='\t', header=None, compression="gzip")
mk20[3] = "mk20_"+mk20.index.astype(str)
mk20 = mk20[~mk20[0].isin(['chr7_KI270803v1_alt', 'chr8_KI270821v1_alt', 'chr15_KI270850v1_alt', 'chr22_KI270879v1_alt'])]

In [ ]:
#print(mk20.head())
print("Number of peaks: %d" %len(mk20))

In [ ]:
# Downloading hg38 chromosome sizes as required for flankBed
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/hg38.chrom.sizes

**Function for plotting peak count per chromosome and lineplot for chromosome size**

In [ ]:
def peak_count_plot(dataframe):
    df_plot = dataframe.iloc[:,0].value_counts().rename_axis('chromosomes').reset_index(name='counts')
    df_plot['counts'] = np.log(df_plot['counts'])

    chrsizes = pd.read_csv("/home/jovyan/jm_jlab/data_indNeuro/consensus_atlas_ATACregions_hg38/hg38.chrom.sizes", header=None, sep='\t')

    chrsizes.rename(columns={0:"chromosomes", 1:"size"}, inplace=True)

    df_plot = df_plot.merge(chrsizes, how = 'inner', on = ['chromosomes'])
    df_plot['size'] = np.log(df_plot['size'])

    df_plot['chromosomes'] = df_plot['chromosomes'].str.replace(r'chr', '')
    df_plot.sort_values(by=['chromosomes'], inplace=True)
    print(df_plot.head())
    print(len(df_plot.head()))

    chr_list = dataframe.iloc[:,0].value_counts().rename_axis('chromosomes').reset_index(name='counts')['chromosomes'].unique()   
    
    if len(chr_list) < 22:
        raise "Function was expecting at least 22 chromosomes"
    if 'chrX' and 'chrY' in chr_list:
        p1 = sns.barplot(data=df_plot, x="chromosomes", y="size", order=np.arange(1,23).astype(str).tolist()+['X','Y'])
        p1.set_xticklabels(p1.get_xticklabels(), rotation=90)
    if ('chrX' in chr_list) and ('chrY' not in chr_list):
        p1 = sns.barplot(data=df_plot, x="chromosomes", y="size", order=np.arange(1,23).astype(str).tolist()+['X'])
        p1.set_xticklabels(p1.get_xticklabels(), rotation=90)
    if ('chrY' in chr_list) and ('chrX' not in chr_list):
        p1 = sns.barplot(data=df_plot, x="chromosomes", y="size", order=np.arange(1,23).astype(str).tolist()+['Y'])
        p1.set_xticklabels(p1.get_xticklabels(), rotation=90)
    if ('chrY' not in chr_list) and ('chrX' not in chr_list):
        p1 = sns.barplot(data=df_plot, x="chromosomes", y="size", order=np.arange(1,23).astype(str).tolist())
        p1.set_xticklabels(p1.get_xticklabels(), rotation=90)
        
    # get the xtick locations
    xticks = p1.get_xticks()

    # plot the line to the xtick locs (or df.index)
    p2 = sns.lineplot(data=df_plot, x=xticks, y='counts', marker='o', ax=p1)

    p1.set(ylabel='log peak count (bar) or log chr size (line)', xlabel='chromosome')
    plt.show()
      

In [ ]:
peak_count_plot(mk20)

In [ ]:
tr21singl = pd.read_csv("tr21atac.bed.gz", sep='\t', header=None, compression='gzip')
tr21singl = tr21singl.iloc[:,0:3]
tr21singl[3] = "tr21singleome_"+tr21singl.index.astype(str)
tr21mult = pd.read_csv("tr21multiome.txt.gz", sep='\t', compression='gzip')
tr21mult = tr21mult.iloc[:,0:3]
tr21mult['start'] = tr21mult['start']-1 #Must be 0-based
tr21mult[3] = "tr21multiome_"+tr21mult.index.astype(str)

In [ ]:
#print(tr21singl.head())
print("Number of peaks Trevino singleome: %d" %len(tr21singl))
print("")

#print(tr21mult.head())
print("Number of peaks Trevino multiome: %d" %len(tr21mult))

In [ ]:
peak_count_plot(tr21singl)

In [ ]:
peak_count_plot(tr21mult)

In [ ]:
ls

In [ ]:
dT_hg19 = pd.read_excel("./dT.xlsx") #pip install openpyxl
print(dT_hg19.head(2))

In [ ]:
dT_hg19 = dT_hg19.iloc[:,0:3]
dT_hg19['Chr'] = 'chr'+dT_hg19['Chr'].astype(str)
print(dT_hg19.head(2))
dT_hg19['peak_name'] = "dT18_"+dT_hg19.index.astype(str)
print("Number of peaks %d" %len(dT_hg19))

In [ ]:
peak_count_plot(dT_hg19) #chromosome size corresponds to hg38 though, just in case to spot weird things

**Saving bed files**

In [ ]:
tr21singl.to_csv("./trS.bed", sep='\t', header=None, index=False)
tr21mult.to_csv("./trM.bed", sep='\t', header=None, index=False)
mk20.to_csv("./mk20.bed", sep='\t', header=None, index=False)
dT_hg19.to_csv("./dT18_hg19.bed", sep='\t', header=None, index=False)

In [ ]:
for item in ['tr21multiome.txt.gz', 'tr21atac.bed.gz', 'mk20ocr.txt.gz', 'dT.xlsx']:
    os.remove(item)

In [ ]:
ls

In [ ]:
!bedtools sort -i trS.bed > trS_sorted.bed

In [ ]:
!bedtools sort -i trM.bed > trM_sorted.bed

In [ ]:
!bedtools sort -i mk20.bed > mk_sorted.bed

In [ ]:
!bedtools sort -i dT18_hg19.bed > dT18_hg19_sorted.bed

In [ ]:
for item in ['dT18_hg19.bed',  'mk20.bed',  'trM.bed',  'trS.bed']:
    os.remove(item)

In [ ]:
!cat trS_sorted.bed | wc -l
!cat trM_sorted.bed | wc -l
!cat mk_sorted.bed | wc -l
!cat dT18_hg19_sorted.bed | wc -l

**de la Torre-Ubieta hg19 to hg38**

In [ ]:
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg19/liftOver/hg19ToHg38.over.chain.gz

In [ ]:
!rm dT18_unlifted.bed # wc -l: 48
!rm dT18_hg19_sorted.bed

In [ ]:
ls

In [ ]:
!intersectBed -wo -filenames -f 0.5 -a trS_sorted.bed -b trM_sorted.bed mk_sorted.bed dT18_hg38_sorted.bed | uniq | head -n 2 

In [ ]:
# -wa to show original entries; in this case, Trevino singleome
!intersectBed -wa -f 0.5 -a trS_sorted.bed -b trM_sorted.bed mk_sorted.bed dT18_hg38_sorted.bed | uniq | head -n 2

In [ ]:
# Saving (hg38). Trevino singleome ATAC contains the higher number of peaks.
# These were required to overlap at least 50% with either:
# 1) Trevino multiome
# 2) Markenscoff-Papadimitriou ATAC
# 3) de la Torre-Ubieta
!intersectBed -wa -f 0.5 -a trS_sorted.bed -b trM_sorted.bed mk_sorted.bed dT18_hg38_sorted.bed | uniq > trS_intersected_consensus.bed

**concat X, Y chr signals**

In [ ]:
!grep 'chrY\|chrX' mk_sorted.bed > mk_XY.bed
!grep 'chrX' dT18_hg38_sorted.bed > dT18_X.bed

In [ ]:
!cat mk_XY.bed dT18_X.bed > XY_all.bed
!bedtools sort -i XY_all.bed > XY_all_sorted.bed
!cat trS_intersected_consensus.bed XY_all_sorted.bed > consensus_signals.bed

In [ ]:
!cat consensus_signals.bed | head -n 10

In [ ]:
!cat consensus_signals.bed | uniq | wc -l

In [ ]:
!rm *_sorted.bed

!rm dT18_X.bed mk_XY.bed trS_intersected_consensus.bed XY_all.bed

In [ ]:
ls

In [ ]:
#plotting consensus peaks

plot1 = pd.read_csv("/home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/consensus_signals.bed", sep='\t', header=None)

In [ ]:
peak_count_plot(plot1)

**consensus_signals.bed to GREAT web server to build base GRN**

In [ ]:
!mkdir ./GREAT_results

Using GREAT default options (version 4.0.4)
    
    Species Assembly: Human: GRCh38 (UCSC hg38, Dec. 2013)
    Test regions: consensus_signals.bed
    Background regions: Whole genome
    Settings:
         Basal plus extension
             Proximal: 5 kb upstream, 1 kb downstream, plus Distal: 1000 up to kb
         Included curated regulatory domains 

Files saved as:
    Genomic region -> gene association table --> consensus_peaks_hg38_GREAT.txt
    Gene-genomic region --> consensus_peaks_hg38_GREATallgenes.txt

#### **Regulatory region coordinates from human brain primary samples**

#### *Intersecting with Homo species-specific changes*

**(1) Replicated signals that contain Homo sapiens-derived changes**

In [ ]:
pwd

In [ ]:
# From Kulhwilm and Boeckx (2019)
## Na_arcHFcoords_hg38.bed
## Na_high_freq_hg38.bed

!intersectBed -a /home/jovyan/jm_jlab/martinboeckx_to_hg38/Na_high_freq_hg38.bed -b consensus_signals.bed -wo -f 1 | uniq | wc -l

In [ ]:
!intersectBed -a /home/jovyan/jm_jlab/martinboeckx_to_hg38/Na_high_freq_hg38.bed -b consensus_signals.bed -wo -f 1 > consensus_signals_w_homosapiensSNV.bed

In [ ]:
!cat ./consensus_signals_w_homosapiensSNV.bed | head -n 2 # For enrichment too

In [ ]:
ls

**(2) Extending to 3k bp around variants**

In [ ]:
# consensus_signals_w_homosapiensSNV.bed
# (i) awk: Selecting first four columns (chr, start, end, rsID) and write tmp file as tab-delimited
# (ii) Extending 3000bp around using flankBed (it requires chromosomes' size info)
## If there are two SNPs close enough, the resulting island will be higher than 3000bp

!cat consensus_signals_w_homosapiensSNV.bed | uniq | awk 'BEGIN {OFS="\t"}; { print $1, $2, $3, $4 }' > tmp_consensus_signals_w_homosapiensSNV.bed
!flankBed -i tmp_consensus_signals_w_homosapiensSNV.bed -g hg38.chrom.sizes -b 1500 > tmp2_consensus_signals_w_homosapiensSNV.bed

!rm tmp_consensus_signals_w_homosapiensSNV.bed

!sortBed -i tmp2_consensus_signals_w_homosapiensSNV.bed > tmp2_consensus_signals_w_homosapiensSNV_sorted.bed

!mergeBed -i tmp2_consensus_signals_w_homosapiensSNV_sorted.bed -d 1 > extended_consensus_w_homosapiensSNV_sorted.bed

## Calculate length each interval:
## !mergeBed -i tmp2_consensus_signals_w_homosapiensSNV_sorted.bed -d 1 | awk '{print($3-$2)}' | head

!rm tmp2_consensus_signals_w_homosapiensSNV.bed tmp2_consensus_signals_w_homosapiensSNV_sorted.bed

In [ ]:
!cat extended_consensus_w_homosapiensSNV_sorted.bed | head -n 2

In [ ]:
!cat extended_consensus_w_homosapiensSNV_sorted.bed | wc -l

**(3) Excluding windows that contained Neanderthal/Denisovan variants: Homo sapiens-specific islands**

In [ ]:
# islands with Homo sapiens-derived variants
!intersectBed -wo -v -a extended_consensus_w_homosapiensSNV_sorted.bed -b /home/jovyan/jm_jlab/martinboeckx_to_hg38/Na_arcHFcoords_hg38.bed | wc -l

In [ ]:
!intersectBed -wo -v -a extended_consensus_w_homosapiensSNV_sorted.bed -b /home/jovyan/jm_jlab/martinboeckx_to_hg38/Na_arcHFcoords_hg38.bed | head -n 2

In [ ]:
!intersectBed -wo -v -a extended_consensus_w_homosapiensSNV_sorted.bed -b /home/jovyan/jm_jlab/martinboeckx_to_hg38/Na_arcHFcoords_hg38.bed > islands_HomoSapiens.bed

In [ ]:
!cat islands_HomoSapiens.bed | head

In [ ]:
!cat islands_HomoSapiens.bed | wc -l

In [ ]:
ls

In [ ]:
# Add SNV position to the islands
!intersectBed -wo -a /home/jovyan/jm_jlab/martinboeckx_to_hg38/Na_high_freq_hg38.bed -b islands_HomoSapiens.bed | uniq > islands_HomoSapiens_SNVpos.bed

In [ ]:
# Add SNV position to the islands
!intersectBed -wo -v -b /home/jovyan/jm_jlab/martinboeckx_to_hg38/Na_high_freq_hg38.bed -a islands_HomoSapiens.bed | wc -l #as expected

In [ ]:
# extended regions that *do* contain Neanderthal/Denisovan variants
!cat islands_HomoSapiens.bed | uniq | wc -l

In [ ]:
# extended regions that *do* contain Neanderthal/Denisovan variants
!cat islands_HomoSapiens.bed | uniq | head -n 2

In [ ]:
!awk '{print  $0 "\t region_" NR }' islands_HomoSapiens.bed > uniqID_islands_HomoSapiens.bed

In [ ]:
!cat uniqID_islands_HomoSapiens.bed | head -n 2

In [ ]:
!cat uniqID_islands_HomoSapiens.bed | wc -l 

# **Regulatory region-gene association via Genomic Regions Enrichment of Annotations Tool (GREAT)**

In [ ]:
!gzip /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/20221104-public-4.0.4-Qvpddd-hg38-all-region.txt

In [ ]:
!mkdir enrichment_results

In [ ]:
#Output from GREAT (hg38): 20221029-public-4.0.4-lQ7Q3G-hg38-all-region

great_output  = pd.read_csv("./20221104-public-4.0.4-Qvpddd-hg38-all-region.txt.gz", sep='\t', compression='gzip')

In [ ]:
great_output.head()

In [ ]:
great_output = great_output.iloc[:,0:2]
great_output.rename(columns={'# GREAT version 4.0.4':"peak_name", 'Species assembly: hg38':'gene_name'}, inplace=True)

In [ ]:
len(great_output)

In [ ]:
great_output['gene_name'] = great_output['gene_name'].str.replace(r"\(.*?\)", "", regex=True).str.strip().str.split(" , ")

great_output = great_output.explode('gene_name')

great_output = great_output[great_output["gene_name"].str.contains("NONE")==False]

In [ ]:
great_output.head(2)

In [ ]:
print("Number of genes associated to Homo sapiens regulatory islands: %d" %len(great_output['gene_name'].unique()))

In [ ]:
pd.DataFrame(great_output['gene_name'].unique(), columns=["gene_names"]).to_csv("./enrichment_results/uniqID_islands_HomoSapiens_genenames.tsv", sep='\t', index=False)

In [ ]:
pd.DataFrame(great_output['gene_name'].unique(), 
             columns=["gene_names"]).to_csv("/home/jovyan/jm_jlab/CBL_data/indirectNeurogenesis/regulatory_islands/uniqID_islands_HomoSapiens_genenames.tsv", sep='\t', index=False)

In [ ]:
enrichment = gprofiler(great_output['gene_name'].unique(), organism='hsapiens')

In [ ]:
enrichment.to_csv("./enrichment_results/uniqID_islands_HomoSapiens_enrichment.tsv", index=False, sep='\t')

In [ ]:
enrichment.to_csv("/home/jovyan/jm_jlab/CBL_data/indirectNeurogenesis/regulatory_islands/uniqID_islands_HomoSapiens_enrichment.tsv", index=False, sep='\t')

In [ ]:
for i in pd.DataFrame(enrichment['domain'].value_counts()).index:
    print("Module: %s - Top results" %i)
    print("")
    print(enrichment[(enrichment['domain'] == i)].sort_values('p.value').groupby('domain').head(5)['term.name'])
    print("")


# Dataframes

**Islands Homo sapiens**

In [ ]:
df = pd.read_csv("islands_HomoSapiens_SNVpos.bed", sep='\t', header=None)

In [ ]:
df[7] = df[4]+"_"+df[5].astype(str)+"_"+df[6].astype(str)

In [ ]:
df.rename(columns={0:'chr',1:'SNV_start',2:'SNV_end',3:'rsID',4:'Chr',5:'region_start',6:'region_end', 7:'POS_region'},inplace=True)

In [ ]:
df.head(5)

In [ ]:
len(df)

**Islands Homo sapiens**

In [ ]:
island = pd.read_csv("uniqID_islands_HomoSapiens.bed", sep='\t', header=None)

In [ ]:
island[4] = island[0]+"_"+island[1].astype(str)+"_"+island[2].astype(str)

In [ ]:
island.head(2)

In [ ]:
len(island)

In [ ]:
island.rename(columns={0:'chr',1:'region_start',2:'region_end',3:'region_ID', 4:'POS_region'},inplace=True)

In [ ]:
df = island.merge(df, how = 'inner', on = ['POS_region','chr', 'region_start', 'region_end'])

In [ ]:
df = df.loc[:, df.columns != 'Chr']

In [ ]:
df.rename(columns={'region_start':"island_start", 'region_end':"island_end", "POS_region":"POS_island", "region_ID":"island_ID"}, inplace=True) #for consistency

In [ ]:
df.head(2)

**Assigning gene names to each region** (based on GREAT result)

In [ ]:
great_res  = pd.read_csv("./20221104-public-4.0.4-Qvpddd-hg38-all-region.txt.gz", sep='\t', compression='gzip')

great_res = great_res.iloc[:,0:2]

great_res.rename(columns={'# GREAT version 4.0.4':"island_ID", 'Species assembly: hg38':'gene_name'}, inplace=True)


len(great_res)

In [ ]:
len(great_res)-great_res['gene_name'].value_counts()[0] #without NONE

In [ ]:
#Output from GREAT (hg38): 20221029-public-4.0.4-lQ7Q3G-hg38-all-region
great_res  = pd.read_csv("./20221104-public-4.0.4-Qvpddd-hg38-all-region.txt.gz", sep='\t', compression='gzip')

great_res = great_res.iloc[:,0:2]
great_res.rename(columns={'# GREAT version 4.0.4':"peak_name", 'Species assembly: hg38':'gene_name'}, inplace=True)

great_res['gene_name'] = great_res['gene_name'].str.replace(r"\(.*?\)", "", regex=True).str.strip().str.split(" , ")


great_res = great_res.explode('gene_name')

great_res = great_res[great_res["gene_name"].str.contains("NONE")==False]

great_res = great_res.groupby(['peak_name'], as_index=False).agg(', '.join)

great_res.rename(columns={'peak_name':'island_ID'}, inplace=True)

In [ ]:
great_res.head(2)

In [ ]:
len(great_res) #PANDAS PROCESSING DID WORK WELL

In [ ]:
!cat uniqID_islands_HomoSapiens.bed | head -n 2

In [ ]:
uniqID = pd.read_csv("./uniqID_islands_HomoSapiens.bed", header=None, sep='\t') #input GREAT



uniqID[4] = uniqID[0]+"_"+uniqID[1].astype(str)+"_"+uniqID[2].astype(str)
uniqID.rename(columns={0:'chr',1:'island_start',2:'island_end',3:'island_ID', 4:'POS_island'},inplace=True)

uniqID.head(2)

In [ ]:
len(uniqID)

In [ ]:
great_res['island_ID'][0]

In [ ]:
uniqID['island_ID'][0]

In [ ]:
uniqID['island_ID'] = uniqID['island_ID'].str.strip()

In [ ]:
df_great = uniqID.merge(great_res, how = 'outer', on = ['island_ID'])

In [ ]:
df_great.head(2)

In [ ]:
df['island_ID'] = df['island_ID'].str.strip()

In [ ]:
df_great.columns[:-1].to_list()

In [ ]:
df_final = df.merge(df_great, how = 'outer', on = df_great.columns[:-1].to_list())

In [ ]:
df_final.to_csv("hg38_regulatory_islands_genenames.tsv", sep='\t', index=False)

In [ ]:
df_final.to_csv("/home/jovyan/jm_jlab/CBL_data/indirectNeurogenesis/regulatory_islands/hg38_regulatory_islands_genenames.tsv", sep='\t', index=False)

**For intersections**

In [ ]:
pwd

In [ ]:
!mkdir intersections_hg38

In [ ]:
df_final.to_csv("./intersections_hg38/hg38_regulatory_islands.bed", sep='\t', header=None, index=False)

In [ ]:
!cat ./intersections_hg38/hg38_regulatory_islands.bed | head -n 2

# Regulatory islands to hg19

In [ ]:
!mkdir hg38_to_hg19

In [ ]:
!wget --timestamping 'ftp://hgdownload.soe.ucsc.edu/goldenPath/hg38/liftOver/hg38ToHg19.over.chain.gz' -O /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/hg38_to_hg19/hg38ToHg19.over.chain.gz

In [ ]:
!cat ./regulatory_region_results/hg38_regulatory_islands_genenames.tsv | awk 'BEGIN {OFS="\t"} { print $1,$6,$7,$8 }' > ./regulatory_region_results/hg38_regulatory_islands_SNVs.bed

In [ ]:
!cat ./regulatory_region_results/hg38_regulatory_islands_SNVs.bed | awk 'NR>2 {print last} {last=$0}' > ./regulatory_region_results/hg38_regulatory_islands_SNVs_woHeader.bed

In [ ]:
hg19_islands = pd.read_csv("./hg19_uniqID_islands_HomoSapiens.bed", sep='\t', header=None)

In [ ]:
hg19_islands.rename(columns={0:'chr_hg19', 1:'island_start_hg19', 2:'island_end_hg19', 3:'island_ID'}, inplace=True)

In [ ]:
hg38_islands = pd.read_csv("./regulatory_region_results/hg38_regulatory_islands_genenames.tsv", sep='\t')

In [ ]:
hg38_islands.rename(columns={'island_start':'island_start_hg38',
                             'island_end':'island_end_hg38',
                            'SNV_start':'SNV_start_hg38',
                            'SNV_end':'SNV_end_hg38'}, inplace=True)

In [ ]:
rr_islands_hg38 = pd.merge(hg19_islands, hg38_islands, on='island_ID')

In [ ]:
rr_islands_hg38['SNP_number'] = rr_islands_hg38.index.astype(str)

In [ ]:
rr_islands_hg38.columns

In [ ]:
rr_islands_hg38[['chr','SNV_start_hg38','SNV_end_hg38','SNP_number']].to_csv('./regulatory_region_results/hg38_regulatory_islands_SNVs.bed', header=None, sep='\t', index=False)

In [ ]:
hg19_islands_SNVs = pd.read_csv("./regulatory_region_results/hg19_regulatory_islands_SNVs.bed", sep='\t', header=None)

In [ ]:
hg19_islands_SNVs.rename(columns={0:'Chr',1:'SNV_start_hg19',2:'SNV_end_hg19',3:'SNP_number'}, inplace=True)

In [ ]:
hg19_islands_SNVs['SNP_number'] = hg19_islands_SNVs['SNP_number'].astype(str)

In [ ]:
final_genomesVersions = pd.merge(rr_islands_hg38, hg19_islands_SNVs, on='SNP_number')

In [ ]:
final_genomesVersions.drop(columns=['SNP_number'], inplace=True)

In [ ]:
final_genomesVersions.head()

In [ ]:
final_genomesVersions['POS'] = final_genomesVersions['Chr'].str.split("chr", expand=True, n=2)[1].astype(str)+":"+final_genomesVersions['SNV_end_hg19'].astype(str)

In [ ]:
final_hg19hg38 = pd.merge(final_genomesVersions,martin[['POS', 'REF', 'ALT']], on='POS')

In [ ]:
final_hg19hg38.to_csv("./regulatory_region_results/regulatory_islands_completeINFO.tsv", sep='\t', index=False)

# Intersections with evolutionary-relevant regions

**Pey and Akey to hg38**

In [ ]:
pwd

In [ ]:
!intersectBed -wo -f 1 -a /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/hg38_regulatory_islands.bed -b /home/jovyan/jm_jlab/martinboeckx_to_hg38/bed_files/akeydeserts_hg38_sorted.bed | wc -l

In [ ]:
!intersectBed -wo -f 1 -a /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/hg38_regulatory_islands.bed -b /home/jovyan/jm_jlab/martinboeckx_to_hg38/bed_files/akeydeserts_hg38_sorted.bed | head -n 2

In [ ]:
!intersectBed -wo -f 1 -a /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/hg38_regulatory_islands.bed -b /home/jovyan/jm_jlab/martinboeckx_to_hg38/bed_files/akeydeserts_hg38_sorted.bed > /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/hg38_regulatory_islands_DESERTS.tsv

In [ ]:
#adding header

!echo -e 'chr\tisland_start\tisland_end\tisland_ID\tPOS_island\tSNV_start\tSNV_END\trsID\tgene_name\tchr\tDesert_start\tDesert_end\tbp_overlap' | cat - /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/hg38_regulatory_islands_DESERTS.tsv > ./out && mv ./out /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/hg38_regulatory_islands_DESERTS.tsv

In [ ]:
!intersectBed -wo -f 1 -a ./regulatory_region_results/intersections_hg38/hg38_regulatory_islands.bed -b /home/jovyan/jm_jlab/martinboeckx_to_hg38/bed_files/pey_hg38_sorted.bed | wc -l

In [ ]:
!intersectBed -wo -f 1 -a ./regulatory_region_results/intersections_hg38/hg38_regulatory_islands.bed -b /home/jovyan/jm_jlab/martinboeckx_to_hg38/bed_files/pey_hg38_sorted.bed > ./regulatory_region_results/intersections_hg38/hg38_regulatory_islands_POS_Sel.tsv

In [ ]:
#adding header

!echo -e 'chr\tisland_start\tisland_end\tisland_ID\tPOS_island\tSNV_start\tSNV_END\trsID\tgene_name\tchr\tPosSel_start\tPosSel_end\tbp_overlap' | cat - /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/hg38_regulatory_islands_POS_Sel.tsv > ./out && mv ./out /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/hg38_regulatory_islands_POS_Sel.tsv

**Human accelerated regions - HARs (hg38)**

In [ ]:
pwd

In [ ]:
!wget https://www.biorxiv.org/content/biorxiv/early/2022/10/05/2022.10.04.510859/DC1/embed/media-1.xlsx?download=true -O ST1_hg38_zooHARs.xlsx

In [ ]:
#To bed
df = pd.read_excel("/home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_zooHARs.xlsx", sheet_name='zooHARs')

df['start'] = df['start']-1 #must be 0-based

df.iloc[:,0:3].to_csv("/home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_zooHARs.bed", sep='\t', index=False, header=None)

# !cat /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_zooHARs.bed | head

!rm /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_zooHARs.xlsx 

!bedtools sort -i /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_zooHARs.bed > /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_zooHARs_sorted.bed

!rm /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_zooHARs.bed

In [ ]:
!cat ./ST1_hg38_zooHARs_sorted.bed | wc -l # n as expected

In [ ]:
!intersectBed -wo -f 1 -a /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_zooHARs_sorted.bed -b ./regulatory_region_results/intersections_hg38/hg38_regulatory_islands.bed  > ./regulatory_region_results/intersections_hg38/hg38_HARs_overlaps_regulatory_islands.bed

In [ ]:
#adding header

!echo -e 'chr\tHAR_start\tHAR_end\tchr\tisland_start\tisland_end\tisland_ID\tPOS_island\tSNV_start\tSNV_END\trsID\tgene_name\tbp_overlap' | cat - ./regulatory_region_results/intersections_hg38/hg38_HARs_overlaps_regulatory_islands.bed > ./out && mv ./out ./regulatory_region_results/intersections_hg38/hg38_HARs_overlaps_regulatory_islands.tsv

In [ ]:
!rm ./regulatory_region_results/intersections_hg38/hg38_HARs_overlaps_regulatory_islands.bed
!rm ./ST1_hg38_zooHARs_sorted.bed

**human ancestor quickly evolved regions - HAQERs (hg38)**

In [ ]:
pwd

In [ ]:
!wget https://www.cell.com/cms/10.1016/j.cell.2022.10.016/attachment/fed00158-7a2e-4561-b879-29d0b84c2bd2/mmc1.xlsx -O ST1_hg38_HAQERs.xlsx

In [ ]:
#To bed
df = pd.read_excel("/home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_HAQERs.xlsx", sheet_name='HaqerOverlaps')

df['START (hg38)'] = df['START (hg38)']-1 #0-based
df.loc[35,'CHROM (hg38)'] = "chr14" #missing entry

df.iloc[:,1:4].to_csv("/home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_HAQERs.bed", sep='\t', index=False, header=None)

# !cat /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_HAQERs.bed | head

!rm /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_HAQERs.xlsx 

!bedtools sort -i /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_HAQERs.bed > /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_HAQERs_sorted.bed

!rm /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/ST1_hg38_HAQERs.bed

In [ ]:
!cat ./ST1_hg38_HAQERs_sorted.bed | wc -l # n as expected

In [ ]:
!intersectBed -wo -f 1 -a ./ST1_hg38_HAQERs_sorted.bed -b ./regulatory_region_results/intersections_hg38/hg38_regulatory_islands.bed  | wc -l

In [ ]:
!intersectBed -wo -f 1 -a ./ST1_hg38_HAQERs_sorted.bed -b ./regulatory_region_results/intersections_hg38/hg38_regulatory_islands.bed  | head -n 2

In [ ]:
!intersectBed -wo -f 1 -a ./ST1_hg38_HAQERs_sorted.bed -b ./regulatory_region_results/intersections_hg38/hg38_regulatory_islands.bed  > ./regulatory_region_results/intersections_hg38/hg38_HAQERs_overlaps_regulatory_islands.bed

In [ ]:
#adding header

!echo -e 'chr\tHAQER_start\tHAQER_end\tchr\tisland_start\tisland_end\tisland_ID\tPOS_island\tSNV_start\tSNV_END\trsID\tgene_name\tbp_overlap' | cat - ./regulatory_region_results/intersections_hg38/hg38_HAQERs_overlaps_regulatory_islands.bed > ./out && mv ./out ./regulatory_region_results/intersections_hg38/hg38_HAQERs_overlaps_regulatory_islands.tsv

In [ ]:
!rm ./regulatory_region_results/intersections_hg38/hg38_HAQERs_overlaps_regulatory_islands.bed
!rm ./ST1_hg38_HAQERs_sorted.bed

**human gained enhancers - HGEs** do not match in stages considered here

**Copying to CBL_data**

In [ ]:
!ls /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/

In [ ]:
!cp  /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/hg38_* /home/jovyan/jm_jlab/CBL_data/indirectNeurogenesis/regulatory_islands/

**Plots**

In [ ]:
!wc -l /home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/* #some with header

In [ ]:
data_files = os.listdir('./regulatory_region_results/intersections_hg38/')

for filename in data_files:
    if "hg38" in filename:
        print(filename)

In [ ]:
mydir+filename

In [ ]:
data_files = os.listdir('./regulatory_region_results/intersections_hg38/')
mydir = '/home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/intersections_hg38/'

df_list = []

for filename in data_files:
    if "hg38" in filename and "hg38_regulatory_islands.bed" not in filename:
        print(filename)
        df_list.append(pd.read_csv(mydir+filename, sep='\t'))


In [ ]:
#unique number of islands
for i in np.arange(0,len(df_list)):
    print(len(df_list[i].island_ID.unique()))

In [ ]:
d = {'Dataset': ['HS_Positive_selection', 'HS_deserts', 'HARs', 'HAQERs'], 'n regulatory islands': [116, 68, 3, 9]}

In [ ]:
d1 = pd.DataFrame(d)

In [ ]:
sns.barplot(data=d1, x="Dataset", y="n regulatory islands")